Will use rolling statistics from some different categories, according to what ESPN defaults as stats. 

FG%
FT%
3PM
REB
AST
STL
BLK
PTS

In [1]:
import pandas as pd
import numpy as np
FILENAME = 'NBADATA.csv'

def create_rolling_dataset(window,FILENAME,spread=False,winner=False):
    """Uses the dataframe to create rolling averages, in 
        an attempt to capture a team's performance over an N game window prior to the game. 
    """
    
    #retain relevant columns. 
    data = pd.read_csv(FILENAME) 
    del data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['Unnamed: 0'],data['PLUS_MINUS'],data['TOTAL']
    del data['FT'],data['FTA']
    del data['OU']
    del data['OR'], data['DR'],data['PF'] #stuff originally there
    data = data.loc[data['GAME_ID'].values < 41300001] #genius! No playoff games now :)   
    #del data['Team'] 
    #data = pd.get_dummies(data) #sometimes option to hot tcode team, but not yet. Seems like overfitting. 
    teams = data.Team.unique() #each nba team. 
#iterate over those teams, make a rolling window over n games. 
    N_GAMES = window  #different possible rolling window sizes. 
    nba_data = pd.DataFrame([])
    season_ids = []
    for i,val in enumerate(data['GAME_ID'].values):  #loop through every game
        season_ids.append(str(val)[1:3])

    data['Season_ID'] = season_ids #identify the unique seasons. 

    for team in teams:  #for each team
       # print(team)
    #get separate seasons here
        team_data = data.loc[data['Team'] == team]  #this contains the box score of every team game from 2013 to 2018.
        for season in data['Season_ID'].unique(): #this contains the box score of that team for that season. 
            #print(season)
            team_season = team_data.loc[team_data['Season_ID'] == season]
        
            stuff_to_turn_into_avgs = ['TOT', 'ST', 'TO', 'BL', '3P', 'FG%', 'FT%','PTS'] #'OR', 'DR' just removed. 
            for col in team_season.columns:
                if col in stuff_to_turn_into_avgs:
                        team_season['Rolling ' + col] = team_season[col].rolling(window=N_GAMES).mean().shift(1)

            #split each season up here, 
                    #if col != 'PTS':
                    #    team_season['Rolling ' + col] = team_season[col].rolling(window=N_GAMES).mean().shift(1)
                        if col != 'PTS':
                            del team_season[col]
            nba_data =  nba_data.append(team_season)

           # df = pd.concat([road_df,home_df],axis=1)
#reorganize the dataset. 
    nba_data_splits = nba_data.sort_values(by = ['GAME_ID', 'Home','Away'], ascending=[True, True,False])

    nba_data_splits.dropna(inplace=True)  #null values come with rolling means, drop those now. 
    del nba_data_splits['GAME_ID'],nba_data_splits['Date'],nba_data_splits['Home'],nba_data_splits['Away'],nba_data_splits['Team']
    del nba_data_splits['Season_ID']
 
    #delete columns no longer of use, ie team name etc. Can consider keeping team name and see if helps chances. 
    #now align the box scores so its one big one for each game, home team and road teams. 

    road_df = nba_data_splits.iloc[::2]
    home_df = nba_data_splits.iloc[1::2]
    for col in nba_data_splits.columns:
        road_df['road_' + col] = road_df[col]
        home_df['home_' + col] = home_df[col]
    
        del road_df[col],home_df[col]

    home_df.reset_index(inplace=True)
    road_df.reset_index(inplace=True)

#merged into a dataframe here. 
    df = pd.concat([road_df,home_df],axis=1)
    del df['index']

#create the dataset here. Can consider the spread, or winner. 
#at the moment only using a single classifier, that seems sufficient. A home team loss is synonymous with a road team win. 

    df['final_SPREAD'] = df['road_PTS'] - df['home_PTS']
    del df['road_PTS'], df['home_PTS'],df['home_SPREAD']
           # if openspread + endspread <0:
            #    y.append(np.array([0,1,0]))  #home team covered
            #elif openspread + endspread >0:
            #    y.append(np.array([1,0,0]))  #road covered
           # else: 
           #     y.append(np.array([0,0,1]))  #push!
    y = []

    if spread: 
        for i in range(len(df)):
            if df['road_SPREAD'].values[i] + df['final_SPREAD'].values[i] < 0:
                y.append(1) #home team covers
            else: # df['road_SPREAD'].values[i] + df['final_SPREAD'].values[i] > 0:
                y.append(0) #road team covers or push
    #else:
    #    y.append(np.array([0,1]))  #push! 
    
    if winner:
        for i in range(len(df)):
            if df['final_SPREAD'].values[i] < 0: #home team won. 
                y.append(1)
            else:
                y.append(0) #road team won. 

    #del df['final_SPREAD']

#y_names = np.array(['road team win', 'home team win']) #for preprocessing/visualization. 
    X = df
    del X['final_SPREAD']
    return X,y




In [10]:
WINDOW = 5
X, y =create_rolling_dataset(window=WINDOW,FILENAME=FILENAME,winner=True)

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

In [11]:
X['home team win'] = y

In [12]:
X.to_csv('lookback'+str(WINDOW) + 'games_winners' + '.csv')

In [13]:
pd.read_csv('lookback5games_winners.csv')

,Unnamed: 0,road_SPREAD,road_Rolling 3P,road_Rolling TOT,road_Rolling ST,road_Rolling TO,road_Rolling BL,road_Rolling PTS,road_Rolling FG%,road_Rolling FT%,home_Rolling 3P,home_Rolling TOT,home_Rolling ST,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling FG%,home_Rolling FT%,home team win
0,0,6.5,9.4,26.6,9.2,14.4,5.0,113.2,0.477664,0.724035,11.0,28.6,8.6,15.8,3.8,104.8,0.501633,0.771443,1
1,1,13.5,10.2,22.4,6.2,14.8,5.6,100.8,0.416175,0.729286,8.4,17.6,6.8,18.6,4.6,109.4,0.498713,0.712209,0
2,2,3.0,4.2,17.8,7.4,13.0,6.2,90.2,0.421881,0.645434,6.4,16.4,5.2,14.6,3.2,94.0,0.433487,0.702972,0
3,3,-8.5,8.6,21.4,6.0,17.6,10.4,95.4,0.452436,0.747334,4.8,15.8,8.8,18.6,5.4,89.4,0.454401,0.784297,0
4,4,-5.5,8.4,20.0,8.4,16.6,5.4,103.4,0.474791,0.696173,5.8,17.6,10.2,15.2,3.8,90.6,0.410580,0.781355,0
5,5,3.0,7.2,23.8,9.2,19.2,3.8,104.4,0.471733,0.662786,4.0,17.6,6.4,19.4,3.8,90.0,0.416141,0.694815,0
6,6,4.5,9.8,24.0,10.0,15.6,3.6,110.0,0.459247,0.835104,7.0,22.8,10.4,15.4,2.8,102.8,0.406218,0.788802,1
7,7,9.5,10.6,23.0,6.2,15.8,5.6,97.4,0.404113,0.747381,6.4,21.0,9.2,14.0,8.8,96.4,0.432727,0.815500,1
8,8,6.0,12.2,27.8,10.6,20.0,7.2,110.8,0.504002,0.715916,6.4,24.2,7.0,13.8,4.8,99.6,0.475608,0.719524,1
9,9,-2.5,8.6,18.6,9.2,17.4,5.4,97.4,0.457137,0.715909,4.2,16.2,7.8,18.4,3.6,85.0,0.383825,0.689481,0


In [34]:
sum(y)

3349

In [35]:
sum(y)/len(y)

0.5799134199134199

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0)
scaler  = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
from sklearn.neural_network import MLPClassifier


model = MLPClassifier()
model.fit(X_train,y_train)

print("Test: ", model.score(X_train,y_train))
print("score: ", model.score(X_test,y_test))

Test:  0.691295312861
score:  0.684903047091


In [38]:
from sklearn.svm import SVC


model = SVC()
model.fit(X_train,y_train)

print("Test: ", model.score(X_train,y_train))
print("score: ", model.score(X_test,y_test))

Test:  0.693835142
score:  0.686288088643
